# Disc Golf sim
## Use this page to simulate real-world disc flight
### set the inital conditions using the sliders and watch the flight graphs update

In [1]:
from ipywidgets import HBox, VBox, IntSlider, interactive_output, Dropdown
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits import mplot3d

In [2]:
def interactive_refresh_plot(x,y,z):
    plt.figure(1)
    ax = plt.axes(projection ='3d')
    ax.plot3D(x, y, z, 'green')
    plt.figure(2)
    ax = plt.axes(projection ='3d')
    ax.plot3D(x, y, z, 'red')
    ax.view_init(0, 0)
    plt.figure(3)
    ax = plt.axes(projection ='3d')
    ax.plot3D(x, y, z, 'blue')
    ax.view_init(0, 90)
    plt.show()

In [3]:
import frispy
from dataclasses import dataclass

In [4]:
class MccDisc():
    def __init__(self, disc, name):
        self.disc = disc
        self.name = name
@dataclass
class JankieModel(frispy.Model):
    PL0: float = 999
    PLa: float = 999
    PD0: float = 999
    PTy0: float = -999
    PTya: float = 999
@dataclass
class HummelModel(frispy.Model):
    PL0: float = 0.33
    PLa: float = 1.9
    PD0: float = 0.18
    PTy0: float = -0.082
    PTya: float = 0.43
@dataclass
class AviarModel(frispy.Model):
    PL0: float = 0.152
    PLa: float = 0.044
    PD0: float = 0.083
    PTy0: float = -0.018
    PTya: float = 0.002
@dataclass
class BuzzModel(frispy.Model):
    PL0: float = 0.099
    PLa: float = 0.041
    PD0: float = 0.061
    PTy0: float = -0.033
    PTya: float = 0.004
@dataclass
class RocModel(frispy.Model):
    PL0: float = 0.053
    PLa: float = 0.043
    PD0: float = 0.067
    PTy0: float = -0.015
    PTya: float = 0.003
@dataclass
class FlickModel(frispy.Model):
    PL0: float = 0.100
    PLa: float = 0.038
    PD0: float = 0.076
    PTy0: float = -0.007
    PTya: float = 0.008
@dataclass
class StormModel(frispy.Model):
    PL0: float = 0.107
    PLa: float = 0.045
    PD0: float = 0.057
    PTy0: float = -0.026
    PTya: float = 0.004
@dataclass
class WraithModel(frispy.Model):
    PL0: float = 0.143
    PLa: float = 0.040
    PD0: float = 0.055
    PTy0: float = -0.020
    PTya: float = 0.006
@dataclass
class QuarterModel(frispy.Model):
    PL0: float = 0.138
    PLa: float = 0.039
    PD0: float = 0.065
    PTy0: float = -0.038
    PTya: float = 0.005
all_MccDiscs = [
    MccDisc(frispy.Disc(model=frispy.Model()),'none'),
    MccDisc(frispy.Disc(model=JankieModel()),'JankieModel'),
    MccDisc(frispy.Disc(model=HummelModel()),'hummel'),
    MccDisc(frispy.Disc(model=AviarModel()),'putter - aviar'),
    MccDisc(frispy.Disc(model=RocModel()),'mid - roc'),
    MccDisc(frispy.Disc(model=BuzzModel()),'mid - buzz'),
    MccDisc(frispy.Disc(model=WraithModel()),'driver - wraith'),
    MccDisc(frispy.Disc(model=FlickModel()),'driver - flick'),
    MccDisc(frispy.Disc(model=QuarterModel()),'driver - quarter'),
]

In [5]:
def plot_func(a, f):
    #discs = [all_MccDiscs[0]]
    #for mccdisc in discs:
    #    mccdisc.disc.set_default_initial_conditions(vx=23, vy=0, theta=0, phi=0.2)
    #    mccdisc.disc.reset_initial_conditions()
    #results = []
    #for mccdisc in discs:
    #    results.append(mccdisc.disc.compute_trajectory(flight_time=4))
    #x = results[0].x
    #y = results[0].y
    disc = frispy.Disc()
    disc.default_initial_conditions['vx'] = 23+a
    disc.reset_initial_conditions()
    result = disc.compute_trajectory()
    times = result.times
    x, y, z = result.x, result.y, result.z
    interactive_refresh_plot(x,y,z)

In [6]:
a = IntSlider()
b = IntSlider()

In [7]:
out = interactive_output(plot_func, { "a": a, "f": b })

In [8]:
Dropdown(
    options=[('One', 1), ('Two', 2), ('Three', 3)],
    value=2,
    description='Number:',
)

Dropdown(description='Number:', index=1, options=(('One', 1), ('Two', 2), ('Three', 3)), value=2)

In [9]:
display(HBox([VBox([a, b]), out]))